In [185]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
import re
from time import sleep as time_sleep
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED

In [2]:
proxy = {'https': 'http://45.81.76.252:8000'}
proxy_sel = '45.81.76.252:8000'

In [3]:
url = 'https://lenta.com/catalog/'
host = '/'.join(url.split('/', maxsplit=3)[:3])

In [139]:
class initial_driver(NamedTuple):
    browserWidth: int
    browserHeight: int
    browserLeft: int
    browserTop: int
    proxy: str
    start_url: str
    
test = initial_driver(1280, 1300, -2560, 0, '45.81.76.252:8000', 'https://lenta.com/catalog/')

In [140]:
def _start_driver(initial:initial_driver) -> uc.Chrome:
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument(f'--proxy-server={initial.proxy}')
    chrome_options.add_argument(f"--window-position={initial.browserLeft},{initial.browserTop}")
    chrome_options.add_argument(f"--window-size={initial.browserWidth},{initial.browserHeight}")
    driver = uc.Chrome(options=chrome_options)
    driver.get(initial.start_url)
    time_sleep(2)
    return driver




In [44]:
# Сохранение ссылок на категории продуктов
soup = BS(driver.page_source, 'html.parser')

table = soup.find('div', class_='catalog-groups-grid catalog-groups-grid--small-paddings')
link_dict = {}
for item in table.find_all('a'):
    name = item.get('ga-event-label')
    href = host + item.get('href')
    link_dict[name] = href
link_dict

# Сохраняем в JSON файл
with open('data/category_link_dict.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(link_dict, indent=4, ensure_ascii=False))

In [165]:
def select_sity(driver: uc.Chrome):
    wait = WebDriverWait(
        driver,
        timeout=40,
        poll_frequency=1,
        ignored_exceptions=[TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException])

    element = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='address-container__adress-icon-container'))
    
    
    # element = driver.find_element(by=By.CLASS_NAME, value='address-container__adress-icon-container')
    element.click()
    # time_sleep(1)

    element = driver.find_element(by=By.CLASS_NAME, value='store-picker-city__label')
    element = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='store-picker-city__label'))
    element.click()
    # time_sleep(1)

    # elements = driver.find_elements(by=By.CSS_SELECTOR, value="span.cities-list-item__name")
    elements = wait.until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value='span.cities-list-item__name'))
    for el in elements:
        if 'Липецк' in el.text:
            el.click()
            # time_sleep(1)
            break

    # elements = driver.find_elements(by=By.CSS_SELECTOR, value='span.button__inner')
    elements = wait.until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value='span.button__inner'))
    for el in elements:
        if 'Магазины' in el.text:
            el.click()
            # time_sleep(1)
            break

    # elements = driver.find_element(by=By.CLASS_NAME, value='simplebar-content').find_elements(by=By.CLASS_NAME, value="stores-list-item__name")
    elements = wait.until(lambda d: d.find_element(by=By.CLASS_NAME, value='simplebar-content').find_elements(by=By.CLASS_NAME, value="stores-list-item__name"))
    for el in elements:
        if 'ул. Катукова, д. 51' in el.text:
            el.click()
            # time_sleep(1)
            break

    element = driver.find_element(by=By.XPATH, value="//*[contains(text(), 'Выбрать магазин')]")
    element.click()
    time_sleep(5)

In [167]:
driver.quit()

In [115]:
soup = BS(driver.page_source, 'html.parser')

table = soup.find('div', class_='catalog-groups-grid catalog-groups-grid--small-paddings')
link_dict = {}
my_queue = Queue()
for item in table.find_all('a'):
    name = item.get('ga-event-label')
    href = host + item.get('href')
    link_dict[name] = href
    my_queue.put(href)

In [116]:
if my_queue.empty():
    driver.get(url)
else:
    driver.get(my_queue.get())
    time_sleep(1)

In [170]:
def _get_pagination(driver: uc.Chrome) -> int:
    el = driver.find_element(by=By.CLASS_NAME, value='js-catalog-container')
    catalog = json.loads(el.get_attribute('data-catalog-data'))
    pagination = math.ceil(catalog.get('skusCount')/catalog.get('limit'))
    return pagination

In [172]:
def _get_queue_cat(driver: uc.Chrome) -> Queue:
    page_url = driver.current_url
    pagination = _get_pagination(driver)
    my_queue = Queue()
    for key in range(1, pagination+1):
        my_queue.put(f'{page_url}?page={key}')
    return my_queue

In [186]:
def _save_pages(pages_list: list[str], name: str = 'lenta') -> None:
    now = datetime.now()
    formatted_date = now.strftime('%Y-%m-%d_%H-%M')
    with ZipFile(f'data/{formatted_date}-{name}.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
        for index, page in enumerate(pages_list):
            zip_file.writestr(f'{index:05d}.html', page)

In [175]:
with open('data/category_link_dict.json', 'r', encoding='utf-8') as file:
    link_dict = json.loads(file.read())

link_queue = Queue()
for val in link_dict.values():
    link_queue.put(val)

def get_pages(initial_tuple: initial_driver) -> list:
    output_list = []
    
    driver = _start_driver(initial_tuple)
    select_sity(driver)
    count = 0
    while True:
        url = link_queue.get()
        driver.get(url)
        time_sleep(2)
        cat_queue = _get_queue_cat(driver)
        
        while True:
            url = cat_queue.get()
            driver.get(url)
            time_sleep(1)
            output_list.append(driver.page_source)
            
            if cat_queue.empty():
                break
        
        count += 1
        if count == 2:
            break
        if link_queue.empty():
            break
        
    driver.quit()
    return output_list
page_list = get_pages(test)

In [187]:
_save_pages(page_list)